In [15]:
import tensorflow as tf
import pathlib
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import matplotlib.pyplot as plt


In [4]:
# Main directory where images are stored
data_dir = pathlib.Path('defungi')

# Load the full dataset without any split
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=(128, 128),  # Resize to a fixed size
    batch_size=None,  # Load as individual elements
    color_mode='grayscale',  # Convert to grayscale
    seed=42
)

# Separate images and labels
images, labels = [], []
for img, label in dataset:
    images.append(img)
    labels.append(label)

images = np.array(images)  # Convert to numpy arrays
labels = np.array(labels)


Found 9114 files belonging to 5 classes.


In [9]:
images = images / 255.0  # Normalize pixel values to [0, 1]
images = images.reshape(-1, 128, 128, 1)  # Add channel dimension for grayscale

In [11]:
# 60% Train, 40% Temp (to split into validation and test)
x_train, x_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.4, random_state=42)

# 50% Validation, 50% Test from the temp set (20% each of the total data)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)


In [13]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    
    layers.Dense(5, activation='softmax')  # 5 classes for output
])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,709 (12.61 MB)

 Trainable params: 3,304,709 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
learning_rate = 0.00005

optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
#optimizer = keras.optimizers.SGD(learning_rate=learning_rate,momentum=0.9)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 20
history = model.fit(x_train, y_train, epochs=epochs,batch_size=128, validation_data=(x_val, y_val))


Epoch 1/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 125s 3s/step - accuracy: 0.4381 - loss: 1.4547 - val_accuracy: 0.4723 - val_loss: 1.3510
Epoch 2/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 109s 3s/step - accuracy: 0.4917 - loss: 1.3649 - val_accuracy: 0.4723 - val_loss: 1.3487
Epoch 3/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 111s 3s/step - accuracy: 0.4904 - loss: 1.3526 - val_accuracy: 0.4723 - val_loss: 1.3089
Epoch 4/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.4887 - loss: 1.3114 - val_accuracy: 0.4805 - val_loss: 1.2529
Epoch 5/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.5091 - loss: 1.2629 - val_accuracy: 0.4717 - val_loss: 1.2192
Epoch 6/20
32/43 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5121 - loss: 1.2224

In [ ]:
# Plot training and validation loss VS epochs
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.show()


In [ ]:
# Evaluate on the test dataset
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Predict class labels on the test dataset
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = y_test  # y_test already contains the true labels

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Calculate precision and recall
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
